In [1]:
import os
import math
import wandb
import random
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt


from tensorflow.keras.preprocessing.image import load_img

from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.losses import *
from tensorflow.keras.metrics import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.activations import *

from tensorflow.keras.regularizers import *

from tensorflow.keras.callbacks import *
from keras.preprocessing.image import *
from tensorflow.keras.preprocessing import *

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
2023-04-21 16:12:23.219087: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-21 16:12:23.322575: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-21 16:12:23.348555: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory:

In [2]:
# Load the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# Normalize pixel values between 0 and 1
x_train = x_train / 255.0
x_test = x_test / 255.0

# Convert labels to one-hot encoded vectors
y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)

In [3]:
# SERLU 함수 정의
def serlu(x, lamda = 1.0786, alpha= 2.90427):
    return tf.where(x>=0, x, alpha * x * tf.exp(x))

In [4]:
inputs = Input(shape=(32,32, 3))

# Layer 1
x = Conv2D(32, (3,3), activation=serlu, kernel_regularizer=l2(1e-6))(inputs)

# Layer 2
x = Conv2D(32, (3,3), activation=serlu, kernel_regularizer=l2(1e-6))(x)
x = MaxPooling2D((2, 2))(x)
x = Dropout(0.1)(x)

# Layer 3
x = Conv2D(64, (3,3), activation=serlu, kernel_regularizer=l2(1e-6))(x)

# Layer 4
x = Conv2D(64, (3,3), activation=serlu, kernel_regularizer=l2(1e-6))(x)
x = MaxPooling2D((2, 2))(x)
x = Dropout(0.1)(x)

# Layer 5
x = AveragePooling2D()(x)
x = Flatten()(x)
x = Dense(512, activation=serlu, kernel_regularizer=l2(1e-6))(x)

# Layer 6
outputs = Dense(10, activation='softmax')(x)
model = Model(inputs=inputs, outputs=outputs)

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 30, 30, 32)        896       
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 14, 14, 32)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 64)        18496     
                                                             

2023-04-21 16:12:25.790879: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-21 16:12:26.460652: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22277 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:68:00.0, compute capability: 8.6


In [5]:
# 모델 컴파일
opt = RMSprop(learning_rate=1e-4)

model.compile(optimizer=opt,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [7]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(horizontal_flip=True, vertical_flip=True)
datagen.fit(x_train)

In [9]:
from tensorflow.keras import callbacks

wandb.init(project="CIFAR10", entity="hcim", name='LeNet-serlu')

wandbCallback=wandb.keras.WandbCallback(monitor="val_accuracy", mode="max", 
                                        log_weights=True, log_gradients=True, 
                                        training_data=datagen.flow(x_train, y_train, batch_size=128))
        
    
callbacks = [wandbCallback]

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: chan4im (hcim). Use `wandb login --relogin` to force relogin


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


In [10]:
model.fit(datagen.flow(x_train, y_train, batch_size=128),
          validation_data=(x_test, y_test),
          epochs=200, workers=4,
          callbacks=callbacks)


Epoch 1/200


2023-04-21 16:14:27.974729: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2023-04-21 16:14:29.256600: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


388/391 [============================>.] - ETA: 0s - loss: 1.9138 - accuracy: 0.2988

INFO:tensorflow:Assets written to: /root/jupyter/IHC/gain_proj/wandb/run-20230421_161424-h4u35zu2/files/model-best/assets


INFO:tensorflow:Assets written to: /root/jupyter/IHC/gain_proj/wandb/run-20230421_161424-h4u35zu2/files/model-best/assets
wandb: Adding directory to artifact (/root/jupyter/IHC/gain_proj/wandb/run-20230421_161424-h4u35zu2/files/model-best)... Done. 0.0s


...

391/391 [==============================] - 19s 49ms/step - loss: 0.5764 - accuracy: 0.7967 - val_loss: 0.7581 - val_accuracy: 0.7365
Epoch 175/200
391/391 [==============================] - 17s 45ms/step - loss: 0.5739 - accuracy: 0.8000 - val_loss: 0.7673 - val_accuracy: 0.7327
Epoch 176/200
391/391 [==============================] - 18s 45ms/step - loss: 0.5746 - accuracy: 0.7967 - val_loss: 0.7578 - val_accuracy: 0.7358
Epoch 177/200
391/391 [==============================] - ETA: 0s - loss: 0.5745 - accuracy: 0.7975

INFO:tensorflow:Assets written to: /root/jupyter/IHC/gain_proj/wandb/run-20230421_161424-h4u35zu2/files/model-best/assets


INFO:tensorflow:Assets written to: /root/jupyter/IHC/gain_proj/wandb/run-20230421_161424-h4u35zu2/files/model-best/assets
wandb: Adding directory to artifact (/root/jupyter/IHC/gain_proj/wandb/run-20230421_161424-h4u35zu2/files/model-best)... Done. 0.0s


391/391 [==============================] - 19s 48ms/step - loss: 0.5745 - accuracy: 0.7975 - val_loss: 0.7511 - val_accuracy: 0.7390
Epoch 178/200
391/391 [==============================] - 17s 44ms/step - loss: 0.5710 - accuracy: 0.7999 - val_loss: 0.7578 - val_accuracy: 0.7372
Epoch 179/200
391/391 [==============================] - 18s 45ms/step - loss: 0.5725 - accuracy: 0.7975 - val_loss: 0.7681 - val_accuracy: 0.7332
Epoch 180/200
391/391 [==============================] - 18s 45ms/step - loss: 0.5711 - accuracy: 0.7994 - val_loss: 0.7688 - val_accuracy: 0.7309
Epoch 181/200
391/391 [==============================] - 17s 44ms/step - loss: 0.5711 - accuracy: 0.8000 - val_loss: 0.7861 - val_accuracy: 0.7310
Epoch 182/200
391/391 [==============================] - 17s 45ms/step - loss: 0.5646 - accuracy: 0.8022 - val_loss: 0.7609 - val_accuracy: 0.7311
Epoch 183/200
391/391 [==============================] - 18s 46ms/step - loss: 0.5674 - accuracy: 0.8004 - val_loss: 0.7543 - val_ac

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

